In [ ]:
import yt_dlp
import os
import subprocess
from faster_whisper import WhisperModel

def download_manual_subtitles(url, preferred_langs=('ru', 'en')):
    ydl_opts_probe = {
        'quiet': True,
        'skip_download': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts_probe) as ydl:
        info = ydl.extract_info(url, download=False)
        subtitles = info.get('subtitles', {})
        available_langs = list(subtitles.keys())

        for lang in preferred_langs:
            if lang in subtitles:
                ydl_opts_download = {
                    'quiet': True,
                    'skip_download': True,
                    'writesubtitles': True,
                    'subtitleslangs': [lang],
                    'outtmpl': f'subtitles_{lang}.%(ext)s'
                }
                with yt_dlp.YoutubeDL(ydl_opts_download) as ydl2:
                    ydl2.download([url])
                return True
    return False

def extract_audio_from_url(url, output_audio="audio.wav"):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'temp_audio.%(ext)s',
        'quiet': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        audio_file = ydl.prepare_filename(info)

    # Конвертация в WAV
    subprocess.run([
        'ffmpeg', '-y',
        '-i', audio_file,
        '-ar', '16000',
        '-ac', '1',
        output_audio
    ], check=True)

    # Удаляем исходный webm/mp4
    if os.path.exists(audio_file):
        os.remove(audio_file)

    return output_audio


def transcribe_with_whisper(audio_path, output_txt_path="transcript.txt", model_size="base"):
    model = WhisperModel(model_size)
    segments, _ = model.transcribe(audio_path, beam_size=5)

    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write("📄 Расшифровка:\n")
        for seg in segments:
            start = f"{seg.start:.2f}"
            end = f"{seg.end:.2f}"
            text = seg.text.strip()
            f.write(f"[{start} - {end}]  {text}\n")

    # ✅ Только финальное сообщение
    print(f"✅ Расшифровка завершена. Сохранено в {output_txt_path}")
    return output_txt_path



def full_transcription_pipeline(url):
    if download_manual_subtitles(url):
        print("✅ Ручные субтитры скачаны.")
    else:
        print("🔁 Субтитров нет — запускаем Whisper.")
        audio_file = extract_audio_from_url(url)
        transcribe_with_whisper(audio_file)

        # Удаляем .wav после транскрипции
        if os.path.exists(audio_file):
            os.remove(audio_file)


# ▶️ Запуск
url = "https://youtu.be/pqk8QIlRpIo?si=sS427EuETPqERQA2"
full_transcription_pipeline(url)


In [2]:
import yt_dlp
import os
import subprocess


def download_video_audio(url):
    # Опции для видео
    ydl_opts_video = {
        'format': 'bestvideo',
        'outtmpl': 'video.%(ext)s',
        'quiet': True,
    }

    # Опции для аудио
    ydl_opts_audio = {
        'format': 'bestaudio',
        'outtmpl': 'audio.%(ext)s',
        'quiet': True,
    }

    # Скачиваем видео
    with yt_dlp.YoutubeDL(ydl_opts_video) as ydl:
        video_info = ydl.extract_info(url, download=True)
        video_file = ydl.prepare_filename(video_info)

    # Скачиваем аудио
    with yt_dlp.YoutubeDL(ydl_opts_audio) as ydl:
        audio_info = ydl.extract_info(url, download=True)
        audio_file = ydl.prepare_filename(audio_info)

    return video_file, audio_file


def merge_video_audio(video_file, audio_file, output_file):
    # Объединение с помощью subprocess
    subprocess.run([
        'ffmpeg', '-y',  # -y чтобы не спрашивал про перезапись
        '-i', video_file,
        '-i', audio_file,
        '-c:v', 'copy',
        '-c:a', 'aac',
        '-strict', 'experimental',
        output_file
    ], check=True)


if __name__ == '__main__':
    url = "https://youtu.be/pqk8QIlRpIo?si=sS427EuETPqERQA2"
    video_file, audio_file = download_video_audio(url)
    merge_video_audio(video_file, audio_file, "video_with_audio.mp4")
    print("Скачано и объединено:", "video_with_audio.mp4")


Скачано и объединено: video_with_audio.mp4               
